In [72]:
%%javascript
function ClickConnect(){
console.log("Working");
document.querySelector("colab-toolbar-button#connect").click()
}setInterval(ClickConnect,60000)

<IPython.core.display.Javascript object>

In [73]:
!nvidia-smi -L
!nvidia-smi
!lscpu |grep 'Model name'
# !lscpu 
!free -h --si | awk  '/Mem:/{print $2}'

/bin/bash: nvidia-smi: command not found
/bin/bash: nvidia-smi: command not found
Model name:                      Intel(R) Xeon(R) CPU @ 2.20GHz
12G


In [74]:
!pip install transformers datasets
!pip install transformers
!pip install 'transformers[torch]'
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [75]:
# !conda install -c huggingface transformers

In [76]:
!ls -a ~/.cache/

.  ..  huggingface  matplotlib	node-gyp  pip


In [91]:
from transformers import AutoTokenizer, BertForMaskedLM, BertForSequenceClassification
import torch

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
mmodel = BertForMaskedLM.from_pretrained("bert-base-uncased")
scmodel = BertForSequenceClassification.from_pretrained("bert-base-uncased")

inputs = tokenizer("The capital of France is [MASK].", return_tensors="pt")

with torch.no_grad():
    logits = mmodel(**inputs).logits

# retrieve index of [MASK]
mask_token_index = (inputs.input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]

predicted_token_id = logits[0, mask_token_index].argmax(axis=-1)
predicted_words=tokenizer.decode(predicted_token_id)

print(f"Predicted words: {predicted_words}")

labels = tokenizer("The capital of France is Paris.", return_tensors="pt")["input_ids"]
# mask labels of non-[MASK] tokens
labels = torch.where(inputs.input_ids == tokenizer.mask_token_id, labels, -100)

outputs = mmodel(**inputs, labels=labels)
round(outputs.loss.item(), 2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transf

Predicted words: paris


0.88

In [92]:
outputs

MaskedLMOutput(loss=tensor(0.8752, grad_fn=<NllLossBackward0>), logits=tensor([[[ -6.4346,  -6.4063,  -6.4097,  ...,  -5.7691,  -5.6326,  -3.7883],
         [-14.0119, -14.7240, -14.2120,  ..., -11.6976, -10.7304, -12.7617],
         [ -9.6561, -10.3125,  -9.7459,  ...,  -8.7782,  -6.6036, -12.6596],
         ...,
         [ -3.7861,  -3.8572,  -3.5644,  ...,  -2.5593,  -3.1093,  -4.3820],
         [-11.6598, -11.4274, -11.9266,  ...,  -9.8772, -10.2103,  -4.7594],
         [-11.7267, -11.7509, -11.8040,  ..., -10.5943, -10.9407,  -7.5151]]],
       grad_fn=<ViewBackward0>), hidden_states=None, attentions=None)

In [96]:
mmodel, scmodel

(BertForMaskedLM(
   (bert): BertModel(
     (embeddings): BertEmbeddings(
       (word_embeddings): Embedding(30522, 768, padding_idx=0)
       (position_embeddings): Embedding(512, 768)
       (token_type_embeddings): Embedding(2, 768)
       (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
       (dropout): Dropout(p=0.1, inplace=False)
     )
     (encoder): BertEncoder(
       (layer): ModuleList(
         (0-11): 12 x BertLayer(
           (attention): BertAttention(
             (self): BertSelfAttention(
               (query): Linear(in_features=768, out_features=768, bias=True)
               (key): Linear(in_features=768, out_features=768, bias=True)
               (value): Linear(in_features=768, out_features=768, bias=True)
               (dropout): Dropout(p=0.1, inplace=False)
             )
             (output): BertSelfOutput(
               (dense): Linear(in_features=768, out_features=768, bias=True)
               (LayerNorm): LayerNorm((768,), ep

In [31]:
predicted_token_id

tensor([3000])

In [30]:
tokenizer.decode(predicted_token_id)

'paris'

In [57]:
inputs

{'input_ids': tensor([[ 101, 1996, 3007, 1997, 2605, 2003,  103, 1012,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [58]:
logits.shape

torch.Size([1, 9, 30522])

In [59]:
mask_token_index

tensor([6])

In [60]:
inputs["input_ids"][0]

tensor([ 101, 1996, 3007, 1997, 2605, 2003,  103, 1012,  102])

In [61]:
words = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])

In [62]:
words

['[CLS]', 'the', 'capital', 'of', 'france', 'is', '[MASK]', '.', '[SEP]']

In [69]:
labels

tensor([[-100, -100, -100, -100, -100, -100, 3000, -100, -100]])

In [70]:
from transformers import pipeline

fill_masker = pipeline(model="bert-base-uncased")
fill_masker("This is a simple [MASK].")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'score': 0.04227164387702942,
  'token': 3291,
  'token_str': 'problem',
  'sequence': 'this is a simple problem.'},
 {'score': 0.031049933284521103,
  'token': 3160,
  'token_str': 'question',
  'sequence': 'this is a simple question.'},
 {'score': 0.029722534120082855,
  'token': 8522,
  'token_str': 'equation',
  'sequence': 'this is a simple equation.'},
 {'score': 0.026522196829319,
  'token': 2028,
  'token_str': 'one',
  'sequence': 'this is a simple one.'},
 {'score': 0.024157337844371796,
  'token': 3627,
  'token_str': 'rule',
  'sequence': 'this is a simple rule.'}]

In [88]:
classifier = pipeline("text-classification", model=mmodel, tokenizer=tokenizer)  #"fill-mask"
sentences = [
    "The judge told the jurors to think fast.\n And I had to go straight to the point.",
    "The judge told that the jurors to think fast."
]
classifier(sentences)

[{'label': 'LABEL_1', 'score': 0.5505396723747253},
 {'label': 'LABEL_1', 'score': 0.5584419369697571}]